In [ ]:
subscription_key = '65733df168e2404583e373ba1df91165'
assert subscription_key

vision_base_url = 'https://uksouth.api.cognitive.microsoft.com/'
detect_url = vision_base_url + 'face/v1.0/detect'

In [ ]:
source_faceID = 'fdf873c6-0e0b-4f56-b948-9ef1d369670b'
verify_url = vision_base_url + 'face/v1.0/verify'
FaceIdentical = 'true'
source_dir = "Images/"

In [ ]:
import requests
params_detect = {
    'returnFaceId': 'true', 
    'returnFaceLandmarks': 'true',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
    }

In [ ]:
%matplotlib inline
#%matplotlib qt - inline for inline; qt is for new windows
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
from io import BytesIO

In [ ]:
def local_image(image_locn, type):
    if type == "local":
        image_data = open(image_locn, "rb").read()
        headers = {'Ocp-Apim-Subscription-key': subscription_key, 
                   'Content-Type': 'application/octet-stream'}
        response = requests.post(detect_url, headers=headers, params=params_detect, data=image_data)
    else:
        headers = {'Ocp-Apim-Subscription-key': subscription_key }
        response = requests.post(detect_url, headers=headers, params=params_detect, json={"url": image_locn})

    faces = response.json()
    
#    print(faces)
    
    if type == "local":
        image_file = image_locn
    else:
        image_file = BytesIO(requests.get(image_locn).content)    

    image = Image.open(image_file)
    
    plt.figure(figsize=(8,8))
    ax=plt.imshow(image, alpha=0.6)
    for face in faces:
        fr = face["faceRectangle"]
        fa = face["faceAttributes"]
        origin = (fr["left"], fr["top"])
        
#        print(face["faceId"])

        FaceIdentical = verify(face["faceId"])

        #take the faceId and compare to the source_faceId - if true then put a green line around the face, if fales then put a red line around the face
        
        if FaceIdentical == 'true':
            p = patches.Rectangle(origin, fr["width"], fr["height"], fill=False, linewidth=2, color='g')
        else:
            p = patches.Rectangle(origin, fr["width"], fr["height"], fill=False, linewidth=2, color='r')
        
        ax.axes.add_patch(p)
        plt.text(origin[0], origin[1], "%s, %d"%(fa["gender"].capitalize(), fa["age"]), fontsize=20, weight="bold", va = "bottom")

In [ ]:
def verify(another_face_id):
    headers = {'Ocp-Apim-Subscription-key': subscription_key }
    json_call = {'faceId1': source_faceID,
            'faceId2': another_face_id}
    response = requests.post(verify_url, headers=headers, json=json_call)
    
    validate = response.json()
#    print(validate)
    
    if validate["isIdentical"] == False:
        return('false')
    else:
        return('true')

In [ ]:
import os
for root, dirs, files in os.walk(source_dir):
    for filename in files:
        temp_filename = source_dir + filename
#        print(temp_filename)
        local_image(temp_filename, "local")

In [ ]:
#local_image("Images/TestImage001.jpg", "local")

In [ ]:
#local_image("https://how-old.net/Images/faces2/main001.jpg", "web")